In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import pathlib
import sys

import pandas as pd
from cev.widgets import Embedding, EmbeddingComparisonWidget

data_dir = pathlib.Path.cwd() / ".." / ".." / "data" / "mair-2022-ismb"
name = "TISSUE_138_samples_FM96_OM138_035_CD45_live_fcs_110595"

### Ozette vs Regular

In [17]:
a = Embedding.from_ozette(pd.read_parquet(data_dir / f"{name}_umap.parquet"))
b = Embedding.from_ozette(pd.read_parquet(data_dir / f"{name}_umap_annotated.parquet"))

In [19]:
widget = EmbeddingComparisonWidget(a, b, titles=["UMAP", "Annotation-transformed UMAP"])
widget

EmbeddingComparisonWidget(children=(VBox(children=(MarkerSelectionIndicator(active=[True, True, True, True, Tr…